In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from src.labeling import get_daily_vol, get_events, get_bins

In [2]:
# 1. Charger les données traitées (processed)
df = pd.read_parquet('../data/processed/train_stationary.parquet')

In [3]:
# FORCE l'index à être un entier
df.index = df.index.astype(int)

# Vérifie que t1 est aussi propre
t1 = df.index.searchsorted(df.index + 5)
t1 = t1[t1 < df.shape[0]]
t1 = pd.Series(df.index[t1], index=df.index[:t1.shape[0]])
# FORCE t1 à être entier (normalement c'est auto, mais on blinde)
t1 = t1.astype(int)

In [4]:
# Pour les calculs de barrière, on a besoin d'un proxy de "Prix".
# Comme Hull Tactical prédit le S&P500, on peut reconstruire un indice synthétique
# à partir des rendements fournis ('forward_returns').
# Formule: Prix_t = Prix_t-1 * (1 + ret)
df['synthetic_price'] = (1 + df['forward_returns']).cumprod()

In [5]:
# 2. Calculer la Volatilité Dynamique (Snippet 3.1)
# C'est ce qui va définir l'épaisseur de nos barrières.
daily_vol = get_daily_vol(df['synthetic_price'], span0=50)

In [6]:
# 3. Définir les Barrières Verticales (Temps)
# La compétition demande une prévision à 1 jour (daily returns).
# Mais pour l'entraînement, on peut se donner un peu d'air.
# Disons qu'on laisse 5 jours max au trade pour réussir.
t1 = df.index.searchsorted(df.index + 5) # +5 index (jours)
t1 = t1[t1 < df.shape[0]]
t1 = pd.Series(df.index[t1], index=df.index[:t1.shape[0]])

In [7]:
# 4. Meta-Labeling : Définir un Modèle Primaire
# Pour cet exemple, prenons un signal très simple comme modèle primaire :
# "Si le rendement 'Market Excess' d'hier était positif, on parie à la hausse".
# (Dans la réalité, tu peux utiliser une moyenne mobile ici).
primary_signal = np.sign(df['market_forward_excess_returns'])
primary_signal = primary_signal.replace(0, 1) # Pas de neutre

In [8]:
# 5. Générer les Événements (Triple Barrier)
# pt_sl = [1, 1] -> On prend profit à 1x la volatilité, on coupe à 1x la volatilité.
# min_ret = 0.005 -> On ignore les situations où la volatilité est < 0.5% (bruit).
events = get_events(
    close=df['synthetic_price'],
    t_events=df.index, 
    pt_sl=[1, 1], 
    trgt=daily_vol, 
    min_ret=0.005, 
    t1=t1, 
    side=primary_signal
)

In [9]:
# 6. Obtenir les Labels Finaux
labels = get_bins(events, df['synthetic_price'])

print("Distribution des Labels (0 = Échec du modèle primaire, 1 = Succès):")
print(labels['bin'].value_counts())

Distribution des Labels (0 = Échec du modèle primaire, 1 = Succès):
bin
0.0    963
1.0    954
Name: count, dtype: int64


In [10]:
# Sauvegarder pour l'entraînement
labels.to_parquet('../data/processed/labels.parquet')